In [1]:
# libraries

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
from glob import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from transformers import CLIPProcessor, CLIPModel

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

import seaborn as sns

from tqdm import tqdm
import random

import config
from customCLIP import customCLIP

/usr/local/Caskroom/miniconda/base/envs/EuroSAT/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
customclip = customCLIP()
customclip.train(few_shot=1, mode="coop", save_model=False)


Training coop with 1-shot data...


AttributeError: 'list' object has no attribute 'item'

In [8]:
customclip.prompt_learner

SimplePromptLearner()

In [3]:
FEW_SHOT = 8
MODE = "coop"
IMAGE_PATH = "2750/Forest/Forest_109.jpg"

customclip = customCLIP(model_name="openai/clip-vit-base-patch32")

if MODE == "coop":
    model_path = f"models/prompt_learners/coop/{FEW_SHOT}_shot.pth"
    customclip.load_model(model_path, mode=MODE)
elif MODE in ["linear_probe", "MLP_probe", "logreg_probe"]:
    model_path = f"results/classifiers/{MODE}/{FEW_SHOT}-shot"
    customclip.load_model(model_path, mode=MODE)
elif MODE == "zeroshot":
    pass

customclip.set_testing_mode(MODE)
prediction, prob = customclip.classify_images_clip([IMAGE_PATH])
print(prediction.item(), prob.item())
# print(customclip.class_labels[prediction.item()])

Loaded coop model from models/prompt_learners/coop/8_shot.pth
Testing in coop mode
1 0.9504203796386719
